In [215]:
from propositional import *
import logging
logging.basicConfig(handlers=[logging.FileHandler('log.log', mode='w'),], level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [21]:
from propositional import CNF, BigAnd, BigOr, Interpretation, LiteralFormula, Symbol
from util import *

X = Symbol("X")

In [25]:
@custom_test(BigAnd(BigOr(X)), BigAnd(BigOr(X)))
@custom_test(Neg(BigAnd([X, Symbol("Y")])), BigOr([Neg(X), Neg(Symbol("Y"))]))
@custom_test(Neg(Neg(X)), X)
@custom_test(Neg(Neg(Neg(X))), Neg(X))
def neg_checker(formula):
    if type(formula).__name__ != 'Neg': return formula
    if type(formula.subformula).__name__ == 'BigAnd':
        corr = BigOr([neg_checker(Neg(el)) for el in formula.subformula.subformulae])
    elif type(formula.subformula).__name__ == 'BigOr':
        corr = BigAnd([neg_checker(Neg(el)) for el in formula.subformula.subformulae])
    elif type(formula.subformula).__name__ == 'And':
        corr = BigOr([neg_checker(Neg(formula.subformula.first)), neg_checker(Neg(formula.subformula.second))])
    elif type(formula.subformula).__name__ == 'Or':
        corr = BigAnd([neg_checker(Neg(formula.subformula.first)), neg_checker(Neg(formula.subformula.second))])
    elif type(formula.subformula).__name__ == 'Neg':
        corr = neg_checker(formula.subformula.subformula)
    elif type(formula.subformula).__name__ == 'bool':
        return not formula
    else: return formula
    return corr

In [33]:
@custom_test(BigAnd(X, Symbol("Y")), BigAnd((BigOr(X), BigOr(Symbol("Y")))))
@custom_test(BigAnd(BigOr(X)), BigAnd(BigOr(X)))
@custom_test(BigAnd(Or(X, Symbol("Y"))), BigAnd((X, Symbol("Y"))))
def make_clauses(formula):
    if type(formula).__name__ != "BigAnd": raise TypeError("Unsupported formula type is not BigAnd!!!")
    new = []
    for clause in formula:
        if type(clause).__name__ == 'BigAnd' or type(clause).__name__ == 'And' or type(clause).__name__ == 'Impl': raise TypeError("Unsupported clause type, formula should be in CNF dummy :)")
        elif type(clause).__name__ == 'BigOr': new.append(clause)
        elif type(clause).__name__ == 'Symbol' or type(clause).__name__ == 'Constant' or type(clause).__name__ == 'bool': new.append(BigOr([clause,]))
        elif type(clause).__name__ == "Neg": new.append(BigOr([neg_checker(clause),]))
        elif type(clause).__name__ == "Or": new.append(BigOr((clause.first, clause.second)))
        else: raise TypeError(f"Unknown type: {type(clause).__name__} - should it be handled?")
    nf = BigAnd(new)
    return nf

In [72]:
def get_literals(formula):
    match formula:
        case Symbol(s) | Neg(Symbol(s)):
            yield formula
        case Or(f1, f2) | And(f1, f2):
            yield from get_literals(f1)
            yield from get_literals(f2)
        case Impl(f1, f2):
            yield from get_literals(f1)
            yield from get_literals(f2)
        case BigOr(subforms) | BigAnd(subforms):
            for i in subforms:
                yield from get_literals(i)
        case Constant():
            pass  # Constants are not literals
        case _:
            raise TypeError(f"Unsupported formula type: {type(formula)}")


In [119]:
@custom_test(BigAnd(BigOr(X)), X)
@custom_test(BigAnd((BigOr((Symbol("Y"), Symbol("Z"))), BigOr(X))), X)
def choose_literal(formula: CNF) -> LiteralFormula:
    from random import choice
    clauses = formula.subformulae
    
    # if there is a unit clause, choose that
    for c in clauses:
        if len(list(get_literals(c))) == 1: return next(get_literals(c))
    
    c = choice(clauses)
    lit = choice(list(get_literals(c)))
    return lit

In [184]:
@custom_test(BigAnd(BigOr(X), BigOr(~X)), None)
def dpll(formula: CNF) -> Interpretation | None:
    logger = logging.getLogger(__name__)
    logger.debug(f"Starting dpll on: {formula}")
    interpretation: Interpretation = {}
    formula, interpretation = _dpll(formula, interpretation)
    logger.debug(f"Result: \n\t{formula}\n\t{interpretation}")
    return interpretation
    
    ##
    ### Base case: if the formula is empty, we have a satisfying interpretation
    ##if len(clauses) == 0: return interpretation
    ### Base case: if the formula contains an empty clause, it is unsatisfiable
    ##if any(not clause.subformulae for clause in formula):
    ##    return None
##
    ### Choose a literal from the formula
    ##literal = choose_literal(formula)
##
    ### Recursively try assigning the literal to True
    ##new_formula = BigAnd(BigOr(l for l in clause if l != literal.complement()) for clause in formula if literal not in clause)
    ##new_interpretation = interpretation | _symbol_assignment(literal)
    ##result = dpll(new_formula, new_interpretation)
    ##if result is not None:
    ##    return result
##
    ### Recursively try assigning the literal to False
    ##new_formula = BigAnd(BigOr(l for l in clause if l != literal) for clause in formula if literal.complement() not in clause)
    ##new_interpretation = interpretation | _symbol_assignment(literal.complement())
    ##return dpll(new_formula, new_interpretation)

In [217]:
def _dpll(formula, interpretation):
    logger = logging.getLogger(__name__)
    formula = make_clauses(formula)
    clauses = list(formula.subformulae)
    
    # Base case: if the formula is empty, we have a satisfying interpretation
    if len(clauses) == 0: 
        logger.info(f"\n\tFormula satisfied\n\t\t{formula}\n\t\t{interpretation}")
        return formula, interpretation
    
    # Base case: if the formula contains an empty clause, it is unsatisfiable
    if any(len(c.subformulae) == 0 for c in clauses):
        logger.info(f"\n\tFormula unsatisfiable\n\t\t{formula}\n\t\t{interpretation}")
        return formula, None
    
    # Choose a literal from the formula
    literal = choose_literal(formula)
    logger.info(f"Picked literal: {literal}")

    # Recursively try assigning the literal to True
    new_interpretation = _assign(interpretation, literal, True)
    p1 = [c for c in clauses if literal not in list(get_literals(c)) and neg_checker(Neg(literal)) not in list(get_literals(c))]
    p2 = [list(get_literals(c)) for c in clauses if neg_checker(Neg(literal)) in list(get_literals(c))]
    for c in p2: c.remove(neg_checker(Neg(literal)))
    p1.extend(p2)
    new_formula = BigAnd([BigOr(c) for c in p1])
    res_formula, res_interpretation = _dpll(new_formula, new_interpretation)
    return res_formula, res_interpretation
    
##
##    # Recursively try assigning the literal to False
##    new_interpretation = _assign(interpretation, literal, False)
##    p1 = [c for c in clauses if literal not in list(get_literals(c)) and neg_checker(Neg(literal)) not in list(get_literals(c))]
##    p2 = [list(get_literals(c)) for c in clauses if neg_checker(Neg(literal)) in list(get_literals(c))]
##    for c in p2: c.remove(neg_checker(Neg(literal)))
##    p1.extend(p2)
##           
##    new_formula = BigAnd(BigOr(l for l in clause if l != literal) for clause in formula if literal.complement() not in clause)
##    new_interpretation = interpretation | _symbol_assignment(literal.complement())
##    return dpll(new_formula, new_interpretation)

In [189]:
def _assign(interpretation, literal, val):
    if type(literal).__name__ == 'Neg': ni = interpretation | {literal.subformula: not val}
    if type(literal).__name__ == 'Symbol': ni = interpretation | {literal: val}
    return ni   

In [226]:
f = BigAnd(Formula.parse("(X \\/ Y)"), Formula.parse("(X \\/ ~Y)"), Formula.parse("(Z \\/ ~Z)"))
f1 = BigAnd(BigOr(X))
f2 = BigAnd(BigOr(X), BigOr(~X))
f3 = BigAnd(BigOr(X), BigOr(~X), Formula.parse("(X \\/ Y)"), Formula.parse("(X \\/ ~Y)"))
f4 = BigAnd(BigOr(~X), BigOr((~X, Symbol("Y"), Symbol("Z"))), Formula.parse("(X \\/ Y)"), Formula.parse("(X \\/ ~Y)"), X, Neg(Symbol("Y")))
f7 = BigAnd(BigOr(~X))

In [ ]:
f5 = BigAnd((BigOr(X), BigOr(~X), Formula.parse("(X \\/ Y)"), Formula.parse("(X \\/ ~Y)"), BigOr()))
f6 = BigAnd((BigOr((X, Symbol("Y"), Symbol("Z"))), BigOr((Neg(Symbol("Y")), Symbol("Z"))), BigOr(X, Neg(Symbol("Z")))))
f11 = BigAnd(BigOr(Symbol("X"), Symbol("Y"), Symbol("Z")), BigOr(Neg(Symbol("Y")), Symbol("Z")), BigOr(Symbol("X"), Neg(Symbol("Z"))))
f12 = BigAnd(BigOr(Symbol("A"), Symbol("B"), Symbol("X")), BigOr(Neg(Symbol("B")), Symbol("X")), BigOr(Symbol("A"), Neg(Symbol("X"))))


In [342]:
# gpt generated
f11 = BigAnd(BigOr(Symbol("X"), Symbol("Y"), Symbol("Z")), BigOr(Neg(Symbol("Y")), Symbol("Z")), BigOr(Symbol("X"), Neg(Symbol("Z"))))
f12 = BigAnd(BigOr(Symbol("A"), Symbol("B"), Symbol("X")), BigOr(Neg(Symbol("B")), Symbol("X")), BigOr(Symbol("A"), Neg(Symbol("X"))))
f13 = BigAnd(BigOr(Symbol("Y"), Symbol("Z"), Symbol("A")), BigOr(Neg(Symbol("Z")), Symbol("A")), BigOr(Symbol("Y"), Neg(Symbol("A"))))
f14 = BigAnd(BigOr(Symbol("B"), Symbol("X"), Symbol("Y")), BigOr(Neg(Symbol("X")), Symbol("Y")), BigOr(Symbol("B"), Neg(Symbol("Y"))))
f15 = BigAnd(BigOr(Symbol("X"), Symbol("A"), Symbol("B")), BigOr(Neg(Symbol("A")), Symbol("B")), BigOr(Symbol("X"), Neg(Symbol("B"))))

In [349]:
dpll(f12)

NameError: name 'logging' is not defined

In [352]:
from assignment import dpll

In [351]:
dpll(f6)

NameError: name 'logging' is not defined